In [1]:
# to handle paths
import sys
import os
from pathlib import Path

# to handle datasets
import pandas as pd
import numpy as np
from natsort import natsorted

# for iterating
from itertools import product

# for loading/saving objects
import joblib

# to visualise the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

In [2]:
# variables
project_name = "RF+Clust"
project_folder = f"C:/Users/anani/Downloads/{project_name}"
sys.path.append(project_folder)

from variables import suite_name, features, target, transformation, n_folds, budgets, algorithms
from variables import model_name, explainer_type
from variables import logscale, vmax
from utils import read_fold_data
from rf_clus import RF_clus
from variables import metric, calibration_method, similarity_thresholds, weights_method

In [3]:
# define directories
if features != "all":
    data_folder = f"{project_folder}/Data/{suite_name}/preprocessed/transformation={transformation}_features={features}/{model_name}"
else: 
    data_folder = f"{project_folder}/Data/{suite_name}/preprocessed/transformation={transformation}_features={features}"

results_folder = f"{project_folder}/Results/{suite_name}/transformation={transformation}_features={features}/{model_name}"

## RF+Clus

In [4]:
for algorithm_name, budget, similarity_threshold in product(algorithms, budgets, similarity_thresholds):
    print(f"algorithm_name: {algorithm_name}, budget: {budget}, similarity_threshold: {similarity_threshold}: ")

    # define directories
    data_folder_temp = f"{data_folder}/algorithm_name={algorithm_name}_budget={budget}"
    results_folder_temp = f"{results_folder}/algorithm_name={algorithm_name}_budget={budget}"
    # create directories
    os.makedirs(f"{results_folder_temp}/rfclust", exist_ok=True)

    # create results placeholders
    predictions_rfclus = pd.DataFrame()
    mask = pd.DataFrame()
    similar_instances = pd.DataFrame()
    similarity = pd.DataFrame()

    for fold_number in np.arange(1, n_folds+1):
        print(f"fold_number: {fold_number}")        
                
        # load data
        X_train, y_train, X_test, y_test = read_fold_data(directory=data_folder_temp, fold_number=fold_number) 

        # load trained model
        model = joblib.load(f"{results_folder_temp}/models/model_fold={fold_number}.joblib")

        # load weights
        if weights_method == None:
            weights = None
        else:
            weights = pd.read_csv(f"{results_folder_temp}/rfclust/weights_method={weights_method}.csv", index_col = "fold_number")

            # weights need to have same order as data features
            weights = weights[X_train.columns]
            # get fold weights
            weights = weights[weights.index.get_level_values(0) == fold_number].values[0]
        print(f"weights: ")
        print(weights)

        # init RF_clus
        rf_clus = RF_clus(base_estimator=model
                          , metric=metric
                          , similarity_threshold=similarity_threshold
                          , method=calibration_method
                          , weights=weights
                          )
        # fit on train
        rf_clus.fit(X_train, y_train)

        # predict
        predictions_rfclus_temp, mask_temp, similarity_temp, similar_instances_temp = rf_clus.predict_rf_clust(X_test=X_test
                                                                            , y_test=y_test)

        # concat results for all folds
        predictions_rfclus = pd.concat([predictions_rfclus, predictions_rfclus_temp], axis=0)
        mask = pd.concat([mask, mask_temp], axis=0)
        similar_instances = pd.concat([similar_instances, similar_instances_temp], axis=0)
        similarity = pd.concat([similarity, similarity_temp], axis=0)    
                
    # save
    title = f"metric={metric}_threshold={similarity_threshold}_method={calibration_method}_weights={weights_method}"
    
    print("Predictions RF+Clust:")
    print(predictions_rfclus.head())
    predictions_rfclus.reset_index().to_csv(f"{results_folder_temp}/rfclust/predictions_{title}.csv", index=False)
    
    print("Mask: ")
    print(mask.head())
    mask.reset_index().to_csv(f"{results_folder_temp}/rfclust/mask_{title}.csv", index=False)
    
    print("Similar instances: ")
    print(similar_instances.head())
    similar_instances.reset_index().to_csv(f"{results_folder_temp}/rfclust/similar_instances_{title}.csv", index=False)
    
    print("Similarity: ")
    print(similarity.head())
    similarity.columns = [str(col[0]) + "_" + str(col[1]) for col in similarity.columns]
    similarity.reset_index().to_csv(f"{results_folder_temp}/rfclust/similarity_{title}.csv", index=False)

algorithm_name: DE1, budget: 5000, similarity_threshold: 0.5: 
fold_number: 1
weights: 
None
Predicting with RF+Clust: 
Test instance: (1, 1)


C:\Users\anani\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\anani\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\anani\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results

     disp.diff_mean_02  disp.diff_mean_05  disp.diff_mean_10  \
1 1           0.045757           0.012506          -0.064187   

     disp.diff_mean_25  disp.diff_median_02  disp.diff_median_05  \
1 1          -0.100335             0.054021             0.002367   

     disp.diff_median_10  disp.diff_median_25  disp.ratio_mean_02  \
1 1            -0.080727            -0.123991            0.045821   

     disp.ratio_mean_05  disp.ratio_mean_10  disp.ratio_mean_25  \
1 1            0.012275           -0.064084           -0.099857   

     disp.ratio_median_02  disp.ratio_median_05  disp.ratio_median_10  \
1 1              0.054438               0.00267             -0.080183   

     disp.ratio_median_25  ela_distr.kurtosis  ela_distr.number_of_peaks  \
1 1             -0.124058           -0.197387                  -0.327055   

     ela_distr.skewness  ela_level.lda_mda_10  ela_level.lda_mda_25  \
1 1           -0.227511              3.895617              1.852943   

     ela_level.ld

C:\Users\anani\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\anani\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\anani\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results

weights: 
None
Predicting with RF+Clust: 
Test instance: (4, 1)
     disp.diff_mean_02  disp.diff_mean_05  disp.diff_mean_10  \
4 1          -1.100528          -1.016177          -0.979158   

     disp.diff_mean_25  disp.diff_median_02  disp.diff_median_05  \
4 1          -0.870216             -1.10832            -1.033951   

     disp.diff_median_10  disp.diff_median_25  disp.ratio_mean_02  \
4 1            -0.985625            -0.891411           -1.100682   

     disp.ratio_mean_05  disp.ratio_mean_10  disp.ratio_mean_25  \
4 1           -1.016223            -0.97923           -0.870383   

     disp.ratio_median_02  disp.ratio_median_05  disp.ratio_median_10  \
4 1             -1.108259             -1.034022             -0.985314   

     disp.ratio_median_25  ela_distr.kurtosis  ela_distr.number_of_peaks  \
4 1             -0.891519           -0.196137                  -0.374662   

     ela_distr.skewness  ela_level.lda_mda_10  ela_level.lda_mda_25  \
4 1           -0.202807  

...

C:\Users\anani\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\anani\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\anani\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results

fold_number: 25
weights: 
None
Predicting with RF+Clust: 
Test instance: (25, 1)
      disp.diff_mean_02  disp.diff_mean_05  disp.diff_mean_10  \
25 1           0.297713           0.252903           0.224302   

      disp.diff_mean_25  disp.diff_median_02  disp.diff_median_05  \
25 1           0.157435             0.264505             0.233313   

      disp.diff_median_10  disp.diff_median_25  disp.ratio_mean_02  \
25 1             0.199078             0.124511            0.297645   

      disp.ratio_mean_05  disp.ratio_mean_10  disp.ratio_mean_25  \
25 1            0.252796            0.224772            0.158001   

      disp.ratio_median_02  disp.ratio_median_05  disp.ratio_median_10  \
25 1              0.264653               0.23348              0.199732   

      disp.ratio_median_25  ela_distr.kurtosis  ela_distr.number_of_peaks  \
25 1              0.123777           -0.190284                  -0.184633   

      ela_distr.skewness  ela_level.lda_mda_10  ela_level.lda_mda_2

C:\Users\anani\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\anani\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\anani\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results

weights: 
None
Predicting with RF+Clust: 
Test instance: (30, 1)
      disp.diff_mean_02  disp.diff_mean_05  disp.diff_mean_10  \
30 1           0.464835           0.495582           0.539152   

      disp.diff_mean_25  disp.diff_median_02  disp.diff_median_05  \
30 1           0.658399             0.438706             0.483901   

      disp.diff_median_10  disp.diff_median_25  disp.ratio_mean_02  \
30 1             0.557881             0.663754            0.464866   

      disp.ratio_mean_05  disp.ratio_mean_10  disp.ratio_mean_25  \
30 1            0.495884            0.538873            0.658361   

      disp.ratio_median_02  disp.ratio_median_05  disp.ratio_median_10  \
30 1              0.438945              0.484151              0.557617   

      disp.ratio_median_25  ela_distr.kurtosis  ela_distr.number_of_peaks  \
30 1              0.664011            -0.17467                   0.384193   

      ela_distr.skewness  ela_level.lda_mda_10  ela_level.lda_mda_25  \
30 1       

C:\Users\anani\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\anani\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
